In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [35]:
read_File = spark.sparkContext.wholeTextFiles("example_2.json")

In [42]:
read_File.keys().collect()

[u'file:/C:/Users/DMUKHERJEE/Documents/Debojyoti/Certification/JSON-Practise/example_2.json']

In [43]:
read_File.values().collect()

[u'{\n    "quiz": {\n        "sport": {\n            "q1": {\n                "question": "Which one is correct team name in NBA?",\n                "options": [\n                    "New York Bulls",\n                    "Los Angeles Kings",\n                    "Golden State Warriros",\n                    "Huston Rocket"\n                ],\n                "answer": "Huston Rocket"\n            }\n        },\n        "sport": {\n            "q1": {\n                "question": "5 + 7 = ?",\n                "options": [\n                    "10",\n                    "11",\n                    "12",\n                    "13"\n                ],\n                "answer": "12"\n            },\n            "q2": {\n                "question": "12 - 8 = ?",\n                "options": [\n                    "1",\n                    "2",\n                    "3",\n                    "4"\n                ],\n                "answer": "4"\n            }\n        }\n    }\n}']

In [40]:
read_File_AsDF = spark.read.json(read_File.values())

In [46]:
df_json = spark.read.json("example_2.json",multiLine=True)

In [47]:
df_json.printSchema()

root
 |-- quiz: struct (nullable = true)
 |    |-- maths: struct (nullable = true)
 |    |    |-- q1: struct (nullable = true)
 |    |    |    |-- answer: string (nullable = true)
 |    |    |    |-- options: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- question: string (nullable = true)
 |    |    |-- q2: struct (nullable = true)
 |    |    |    |-- answer: string (nullable = true)
 |    |    |    |-- options: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- question: string (nullable = true)
 |    |-- sport: struct (nullable = true)
 |    |    |-- q1: struct (nullable = true)
 |    |    |    |-- answer: string (nullable = true)
 |    |    |    |-- options: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- question: string (nullable = true)



In [48]:
df_sport = df_json.select("quiz.maths.q1")

In [49]:
df_sport.printSchema()

root
 |-- q1: struct (nullable = true)
 |    |-- answer: string (nullable = true)
 |    |-- options: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- question: string (nullable = true)



In [52]:
df_sport.select("q1.question").show()

+---------+
| question|
+---------+
|5 + 7 = ?|
+---------+



In [3]:
df_movies = spark.read.format("com.databricks.spark.csv").\
                           option("header","true").\
                           option("inferSchema","true").\
                           option("delimiter","~").\
                           load("movies.dat")

In [4]:
df_movies.printSchema()

root
 |-- movieID: integer (nullable = true)
 |-- movieName: string (nullable = true)
 |-- genres: string (nullable = true)



In [6]:
df_movies.createOrReplaceTempView("movies")

In [13]:
df_movies_3_gen = spark.sql("select * from movies where \
                             length(genres) - length(replace(genres,'|',''))=2")

In [17]:
df_movies_3_gen.select("movieName","genres").distinct().show()

+--------------------+--------------------+
|           movieName|              genres|
+--------------------+--------------------+
|Surviving the Gam...|Action|Adventure|...|
|Back to the Futur...|Comedy|Sci-Fi|Wes...|
|        Ronin (1998)|Action|Crime|Thri...|
|Austin Powers: Th...|Action|Adventure|...|
|       D.O.A. (1988)|Film-Noir|Mystery...|
|War and Peace (1956)|   Drama|Romance|War|
|Flight of the Pho...|Action|Adventure|...|
|Same Old Song (On...|Comedy|Drama|Musical|
| Matador, The (2005)|Comedy|Drama|Thri...|
|Sense and Sensibi...|Comedy|Drama|Romance|
|So I Married an A...|Comedy|Romance|Th...|
|  Peeping Tom (1960)|Drama|Horror|Thri...|
|Gold Rush, The (1...|Adventure|Comedy|...|
| For the Boys (1991)|Comedy|Drama|Musical|
|Jurassic Park III...|Action|Adventure|...|
| Split Second (1992)|Action|Sci-Fi|Thr...|
|Man of La Mancha ...|Adventure|Comedy|...|
|Before the Devil ...|Crime|Drama|Thriller|
|Apocalypse Now (1...|    Action|Drama|War|
|     Twilight (1998)|Crime|Dram

In [24]:
df_movies_3_gen.select(df_movies_3_gen["movieName"].alias("MOVIE NAME"),"genres").write.mode("overwrite").json("movies_3_gen")